# An introduction to our Flight Data & Trajectories

## DO NOT MAKE CHANGES OR RUN CODE IN THIS NOTEBOOK

### duplicate this file and call it firstname_lastname_intro.ipynb and do your work there

## Some notes on completing this notebook

- Please do not change anything in the code blocks that is not inside the "YOUR CODE" section
- The 〖value〗inside the brackets is the value that we believe is the correct answer
    - There is a chance we are wrong. If you think we're wrong please let us know
- We know some of these questions will be hard for you. Please ask questions!




## Dataset sneak peek

Let's take a look at the file for this activity. How large is it?

In [ ]:
%%bash

du -h /anvil/projects/tdm/corporate/sandia-trajectory/previous_files/flight/data/raw_data/asdi_2014_07_01_h121314_safe.tsv

#### Question: If this is 3 hours of flight data how much storage does it take for a day? How about a month?

In [ ]:
daysPerMonth = 30
MB_per_GB = 1024

####################         YOUR CODE         ####################

# Report the file size in M
fileSize = 44 

# Get the size of one hour
hourSize = 44/3

# Convert from hours to units of interest
GBperDay = (44/1024)*24
GBperMonth = GBperDay*30

####################         END CODE          ####################

print(f'In one day we collect about {GBperDay:.2f}〖1.03〗GB of flight data!')
print(f'In one month we collect about {GBperMonth:.2f}〖30.94〗GB of flight data!')
print('This dataset actually has a few columns removed to save time on the activty, a month of flight data is around 50 GB.')

#### Question: Let's take a look at the data in our file. How many columns are there? What is stored in each column? What data-type is each column? What units are we using?

In [ ]:
%%bash
head -10 /anvil/projects/tdm/corporate/sandia-trajectory/previous_files/flight/data/raw_data/asdi_2014_07_01_h121314_safe.tsv

#### Fill out this table:

* Column 1 contains the Object Ids of the aircraft, it has type string.
* Column 2 contains the Update_Time in a String
* Column 3 contains the Longitude as a Float.
* Column 4 contains the Lattitude in Float.
* Column 5 contains Speed in integer.
* Column 6 contains Heading in integer.
* Column 7 contains Altitide in integer.

### Data import and manipulation

A quick side-note of time: Its super messy and hard to store. Thankfully the folks over at the [DateTime library](https://docs.python.org/3/library/datetime.html) have put together a handful of useful functions we can make use of.

In [ ]:
from datetime import datetime

BelltowerBirthday = "1995-10-14 12:00:00" #Usally stored in a str
Today = datetime.now()

try: #This wont work
    Today - BelltowerBirthday
except Exception as e: print(e)

# Convert str to DateTime
BelltowerBirthdayDT = datetime.strptime(BelltowerBirthday, '%Y-%m-%d %H:%M:%S')

Age = Today - BelltowerBirthdayDT
print(type(Age))

YearsOld = Age.total_seconds() // (60*60*24*365.25)

print(f'The Purdue Belltower is {YearsOld:.0f} Years Old.')


Bash is cool and all but let's import the file into our favorite languague - Python!

#### Question: Read the file into Pandas and fix any issues with the columns that you read in
*(Hint: file is tab delimiated (.tsv) and we need to convert UPDATE_TIME from str into datetime!)*

In [ ]:
import pandas as pd

fileLocation = '/anvil/projects/tdm/corporate/sandia-trajectory/previous_files/flight/data/raw_data/asdi_2014_07_01_h121314_safe.tsv'

####################         YOUR CODE         ####################

# Read in the file using tab ('\t') seperators
rawFlightData = pd.read_csv(fileLocation, delimiter = '\t')

# Convert the 'UPDATE_TIME' column to datetime
rawFlightData['UPDATE_TIME'] = pd.to_datetime(rawFlightData['UPDATE_TIME'])


####################         END CODE          ####################

print(f'The file for this activty has the shape of {rawFlightData.shape} 〖(808757, 7)〗.')
print(f'(That means it has {rawFlightData.shape[0]} rows, {rawFlightData.shape[1]} columns.)')

#### Question: How many unique idenifiers (object_ids) do we have in this file?

In [ ]:
####################         YOUR CODE         ####################

# Make use of pandas' unique function
uniqueObjectIDs = rawFlightData["OBJECT_ID"].unique()

####################         END CODE          ####################

print(f'There are {len(uniqueObjectIDs)}〖15853〗unique Object IDs.')

#### Question: Let's say that LATITUDE & LONGITUDE are **critial** pieces of information and we can't do our process without them. Please remove rows from the dataset if they are missing **either** of these entries. Call this new dataset `cleanedFlightData`.

In [ ]:
####################         YOUR CODE         ####################

cleanedFlightData = rawFlightData.dropna(subset=['LONGITUDE','LATITUDE'], how='any')

####################         END CODE          ####################

print(f'{rawFlightData.shape[0] - cleanedFlightData.shape[0]}〖29774〗rows were removed.')

#### Question: Let's pick out all rows that use the flight ID `AAL1463` saving it in a variable sharing it's name.

In [ ]:
####################         YOUR CODE         ####################

AAL1463 = cleanedFlightData[cleanedFlightData['OBJECT_ID'] == 'AAL1463']

####################         END CODE          ####################

print(f'There are {len(AAL1463)}〖214〗entires with an OBJECT_ID of AAL40 in our file.')

#### Question: Create a line plot of the flight using `folium.PolyLine`.

In [ ]:
import folium

map = folium.Map()

####################         YOUR CODE         ####################

flightPath = folium.PolyLine(AAL1463[['LATITUDE','LONGITUDE']])

####################         END CODE          ####################

map.add_child(flightPath)

map

### Flight Statistics

Plots are nifty and all but I want to see some stats!

#### Question: Calculate the length of the flight that we chose

In [ ]:
from geopy.distance import geodesic

# Lat,long pairs
carmel = (39.978844, -86.220449)
purdue = (40.426675, -86.926657)
distance = geodesic(carmel, purdue).km

print(f'It is {distance:.2f}km from Carmel to Purdue.')

####################         YOUR CODE         ####################

distanceTraveled = 0
for i in range (len(AAL1463) - 1):
    distanceTraveled += geodesic((AAL1463.iloc[i]['LATITUDE'], AAL1463.iloc[i]['LONGITUDE']), (AAL1463.iloc[i+1]['LATITUDE'], AAL1463.iloc[i+1]['LONGITUDE'])).km


####################         END CODE          ####################

print(f'The aircraft traveled a distance of {distanceTraveled:.2f}〖2004.46〗km.')

#### Question: How long (in seconds) did it take to travel this distance?

In [ ]:
firstPoint = AAL1463["UPDATE_TIME"].iloc[-1]
lastPoint = AAL1463["UPDATE_TIME"].iloc[0]


####################         YOUR CODE         ####################

from datetime import timedelta
timeElapsed = (firstPoint - lastPoint).total_seconds()

####################         END CODE          ####################

print(f'The length of this flight is {timeElapsed:.0f}〖9388〗sconds.')

#### Question: Please tell me the average speed of flight AAL1463 using two methods (average the 'SPEED' column, calculate the distance traveled and divide by time).

In [ ]:
####################         YOUR CODE         ####################
speedAverage = AAL1463['SPEED'].sum()/len(AAL1463)
speedAverageDivided = 2004.46/timeElapsed

####################         END CODE          ####################

print(f'The average speed (via mean) was {speedAverage:.1f}〖413.2〗.')
print(f'The average speed (via dist) was {speedAverageDivided:.3f}〖0.214〗.')
print('Should these values be similar? What are the units of these speeds?')

### Object ID != Flight ID

Each aircraft has a unique Object ID but it may take multiple flights in a given time period. This may skew some of our analysis. Lets take a closer look at one of these examples, `N224JV`.

#### Question: How many points is in N224JV? How long (seconds) is the current "flight"?

In [ ]:
####################         YOUR CODE         ####################

N224JV = cleanedFlightData[cleanedFlightData['OBJECT_ID'] == 'N224JV']
N224JV_timeElapsed = (N224JV["UPDATE_TIME"].iloc[-1] - N224JV["UPDATE_TIME"].iloc[0]).total_seconds()

####################         END CODE          ####################

print(f'N224JV contains {len(N224JV)}〖127〗points.')
print(f'The length of this flight is {N224JV_timeElapsed:.0f}〖8496〗sconds.')

In [ ]:
# Run thise cell to clean up indices
N224JV = N224JV.reset_index()

#### Question: How many seconds occur between each point in the flight?

*Hint: use .diff*, its okay if the first entry is NaN. What is NaN? Why does this occur?

In [ ]:

####################         YOUR CODE         ####################

waitTimes = N224JV['UPDATE_TIME'].diff()

####################         END CODE          ####################

print(waitTimes[0:10])
print(f'The average wait time is {waitTimes.mean().total_seconds():.2f}〖67.43〗seconds.')

#### Question: Lets say an aircraft begins a new flight if we haven't received a datapoint in 30 minutes. At what index do we exceed our temporal gap limit of 30 minutes?

In [ ]:
temporalGapLimit = 30*60

####################         YOUR CODE         ####################

gapIndex = -1
for i in range (len(waitTimes)):
    if waitTimes[i].total_seconds() > temporalGapLimit:
        gapIndex = i
        break

####################         END CODE          ####################

print(f'The temporal gap occurs at an index of {gapIndex}〖43〗.')

#### Question: Split `N224JV` into its two flights `N224JV_1` and `N224JV_2` at the index we found.

In [ ]:
####################         YOUR CODE         ####################

N224JV_1 = N224JV[0:43]
N224JV_2 = N224JV[43:]

####################         END CODE          ####################

print(f'N224JV_1 contains {len(N224JV_1)}〖43〗points.')
print(f'N224JV_2 contains {len(N224JV_2)}〖84〗points.')

#### Question: Plot the results on our map, using a different color for each piece.

In [ ]:
map = folium.Map()

####################         YOUR CODE         ####################

flightPath_1 = folium.PolyLine(N224JV_1[['LATITUDE','LONGITUDE']],color='pink')
flightPath_2 = folium.PolyLine(N224JV_2[['LATITUDE','LONGITUDE']],color='green')

####################         END CODE          ####################
map.add_child(flightPath_1)
map.add_child(flightPath_2)

map

### Wrapping it up
#### Question: Write another summary statistic of a flight (average altitude, speed variance, end-to-end heading) and gets its value for both pieces of `N224JV`.

In [ ]:
# GO CRAZY! RAHHHHH
# I am going crazy!!!

N224JV_1_alt = N224JV_1['ALTITUDE'].sum()/len( N224JV_1)
N224JV_2_alt = N224JV_2['ALTITUDE'].sum()/len( N224JV_2)

print(f'The average altitude of the first flight path is {N224JV_1_alt:.2f}.')
print(f'The average altitude of the second flight path is {N224JV_2_alt:.2f}.')

print('I am so smart 🫡🫡🫡🫡')